In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Load and Check Data

In [ ]:
import os
import warnings
# filter warnings
warnings.filterwarnings('ignore')
from zipfile import ZipFile
print(os.listdir("../input/dogs-vs-cats"))


In [ ]:

ZipFile("/kaggle/input/dogs-vs-cats/train.zip","r").extractall()

ZipFile("/kaggle/input/dogs-vs-cats/test1.zip", "r").extractall()


In [ ]:
train_path = "./train"
test_path = "./test1"

In [ ]:
# Total number of images in train and test datasets
print('Total images in Train and Test Data Set:')
print(len(os.listdir(train_path)))
print(len(os.listdir(test_path)))

In [ ]:
filenames = os.listdir(train_path)
categories = []
for filename in filenames:
    category = filename.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': filenames,
    'category': categories
})
#df train
#if pic is dog pic category=1 else it is cat pic category=0

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
pip install numpy

In [ ]:
pip install Pillow

In [ ]:
import PIL
  
print('Installed Pillow Version:', PIL.__version__)

In [ ]:
df=df.iloc[:10000]

In [ ]:
df.tail()

In [ ]:
from PIL import Image
from numpy import asarray  
import cv2
from PIL import Image, ImageOps
import matplotlib.image as mpimg
from PIL import Image, ImageOps

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

In [ ]:
from PIL import Image
from numpy import asarray  
data=[]
# sample.png is the name of the image
# file and assuming that it is uploaded

# in the current directory or we need
# to give the path
for i in df.filename:
    img=Image.open("./train/"+i)
    img=ImageOps.fit(img,(128,128),Image.ANTIALIAS,0,(0.5,0.5))
    img = ImageOps.grayscale(img)
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    numpydata = asarray(img)
    numpydata=numpydata.flatten()
    data.append(numpydata)

In [ ]:
#data

In [ ]:
type(data)

In [ ]:
x_train=pd.DataFrame(data)


In [ ]:
x_train.head()

In [ ]:
x_train=x_train/255

In [ ]:
x_train


In [ ]:
img

In [ ]:
y_train=df["category"]
y_train

In [ ]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
y_train = to_categorical(y_train, num_classes = 2)

In [ ]:
y_train

In [ ]:
y_train.shape

In [ ]:
x_train=x_train.values.reshape(-1,128,128,1)


print("x_train shape: ",x_train.shape)

In [ ]:
# Split the train and the validation set for the fitting
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=2)
print("x_train shape",x_train.shape)
print("x_val shape",x_val.shape)
print("y_train shape",y_train.shape)
print("y_val shape",y_val.shape)

In [ ]:
#x_train=x_train.values.reshape(-1,128,128,1)


#print("x_train shape: ",x_train.shape)

In [ ]:
# 
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

model = Sequential()
#
model.add(Conv2D(filters = 16, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (128,128,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#
model.add(Conv2D(filters = 8, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))
# fully connected
model.add(Flatten())
model.add(Dense(100, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(2, activation = "sigmoid"))

model.summary()

In [ ]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Compile the model

model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
epochs = 10  # for better result increase the epochs
batch_size = 100

In [ ]:
# data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=5,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
# Fit the model
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_val,y_val), steps_per_epoch=x_train.shape[0] // batch_size)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
import matplotlib.pyplot as plt
plt.plot(history.history['val_loss'], color='b', label="validation loss")
plt.title("Test Loss")
plt.xlabel("Number of Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()